In [6]:
import os
from RISparser import readris
import gensim
from gensim import models
import pprint
from nltk.stem import WordNetLemmatizer
from pprint import pprint

In [2]:
from gensim.parsing.preprocessing import STOPWORDS
stopwords = set(STOPWORDS)
my_stopwords = {'approach','paper','article','model','viewpoint','conclusion','discussion','method','introduction','section','framework','based','result'}
my_stopwords = set.union(stopwords,my_stopwords)
my_stopwords = frozenset(my_stopwords)

In [3]:
data_folder = 'D:/Pesquisa/NLP_LDA_InSySPo/data/'
files = os.listdir(data_folder)

In [4]:
texts = []
dois = []
lem = WordNetLemmatizer()

In [5]:
for ff in files:
    f = data_folder + ff
    fid = open(f,'r',encoding="utf8")
    data = readris(fid)

    for i,entry in enumerate(data):
    
        try:
            title = entry['title']
            abstract = entry['abstract']
            doi = entry['doi']
            dois += [doi]
            text = title + ' ' + abstract
            text = gensim.utils.simple_preprocess(text)
            text = [word for word in text if word not in my_stopwords]
            for i,t in enumerate(text):
                text[i] = lem.lemmatize(t)
            texts += [text]

        except:
            pass



In [ ]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
for index, score in sorted(lda_model[bow_corpus[430]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic ({}): {}".format(score, index, lda_model.print_topic(index, 10)))

In [ ]:
new_doc = 'Piriri pororó'

bow_vector = dictionary.doc2bow(preprocess(new_doc))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))